In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

#Copiando banco de dados do meu Drive para o colab local
!cp '/content/drive/MyDrive/ALLM/DataBaseOrders/STORE_db.DB' '/content'
!cp '/content/drive/MyDrive/ALLM/importDB.py' '/content'
!cp '/content/drive/MyDrive/ALLM/chromaDBconsultClass.py' '/content'
!cp '/content/drive/MyDrive/ALLM/Llama2FineTuned.py' '/content'
!cp '/content/drive/MyDrive/ALLM/ChatGPT.py' '/content'

In [ ]:
!pip install -q langchain sentence_transformers chromadb transformers accelerate einops xformers bitsandbytes faiss-gpu pydantic flask flask-ngrok pyngrok openai

In [49]:
from importDB import CRUDSQLite
from chromaDBconsultClass import ProductsDB
import json
from Llama2FineTuned import FineTunedModel
from ChatGPT import ChamadaGPT

In [50]:
w = ProductsDB()
llm = FineTunedModel()
#llm = ChamadaGPT()

def getResult(userID,query,FirstPrompt):

  db = CRUDSQLite()


  # Consulta pedidos pelo ID do usuario
  resultConsultaDB = db.get_order_by_user_id_jsonFormat(userID)


  # if resultConsultaDB:
  #   print(resultConsultaDB)
  # else:
  #   print("Registro não encontrado.")

  db.close_connection()


  resultadoChromaDB = w.searchDoc(query)
  # print(resultadoChromaDB)
  k = 0
  for i in resultadoChromaDB:
    i =  json.loads(i)
    del i["id"]
    del i["category"]
    del i["Usd-price"]
    del i["sub_category"]
    del i["average_rating"]
    del i["brand"]
    # del i["product_details"]
    # del i["description"]
    i['link'] = i.pop('url')
    resultadoChromaDB[k] = i
    k = k+1

  # prompt = "context: client orders: " + json.dumps(resultConsultaDB) + "\n --- \n estes são os produtos que a loja que você trabalha possue. Indique para o usuário colocando o url: " + str(resultadoChromaDB) + "\n \n " #user question: " + query

  # print(prompt)

  #aqui deve ser feita a consulta para a LLM
  resultadoConsultaLLM = llm.getLLmResponse(json.dumps(resultConsultaDB), str(resultadoChromaDB), query, FirstPrompt)


  return resultadoConsultaLLM
  # return prompt


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [51]:
getResult("b6e18008af6aeb4a5ce5574c15ca2b48","Hello! I'm interested in one of the clothes I saw on your online store, but I'd like to get more information about it. Can you assist me with that?", True)

'===\nPROMPT\n===\n\n<s>[INST] <<SYS>>\n\nPretend to be an online store clerk. Only give products informations that are on store product list or information about customers orders or information about the store politics\n\n<</SYS>>\n\nHello! I\'m interested in one of the clothes I saw on your online store, but I\'d like to get more information about it. Can you assist me with that?\n\nStore product list: [{\'description\': \'Fashion is fun. Shopping for new clothes can be a thrilling experience for a fashion lover looking at latest garments on display is like discovering new treasures. ""What you wear is how you present yourself to the world, especially today, when human contacts are so quick. Fashion is instant language."" You can have anything you want in life if you dress for it. Denim Jacket makes you more attractive to others. Styles Yourself With Trendy Dyed Denim Jackets.\', \'product_details\': \'[{\\\'Color\\\': \\\'Blue\\\'}, {\\\'Fabric\\\': \\\'Denim Blend\\\'}, {\\\'Patter

In [54]:
getResult("b6e18008af6aeb4a5ce5574c15ca2b48","Thank you very much for the information! I feel more confident about purchasing the T-shirt now.", False)

'===\nPROMPT\n===\n\n<s>[INST] <<SYS>>\n\nPretend to be an online store clerk. Only give products informations that are on store product list or information about customers orders or information about the store politics\n\n<</SYS>>\n\nHello! I\'m interested in one of the clothes I saw on your online store, but I\'d like to get more information about it. Can you assist me with that?[/INST]  Hello! Sure, I\'d be happy to help you with more information about the Full Sleeve Printed Men Casual Jacket.\nThe product details you provided are as follows:\n* Color: Brown\n* Fabric: Denim Blend\n* Pattern: Printed\n* Style Code: DJ_Brown\n* Ideal For: Men\n* Sleeve: Full Sleeve\n* Character: None\n* Closure: Button\n* Pack of: 1\n* Suitable For: Western Wear\n* Reversible: No\n* Hooded: No\n* Type: Casual Jacket\n* Fabric Care: Gentle Machine Wash\n* Model Name: "Men\'s Buttoned Flap Pockets Dyed Denim Jacket_Free Size"\n* Generic Name: Jacket\n* Country of Origin: India\n\nAs for your order inf

In [48]:
getResult("b6e18008af6aeb4a5ce5574c15ca2b48","I understand. I'll note the estimated delivery date. Thank you for providing the information.", False)

'===\nPROMPT\n===\n\n<s>[INST] <<SYS>>\n\nPretend to be an online store clerk. Only give products informations that are on store product list or information about customers orders or information about the store politics\n\n<</SYS>>\n\nHello, I would like to get information about my most recent order. Can you help me with that?[/INST]  Hello! Sure, I\'d be happy to help you with your order information. Can you please provide your order ID or customer ID so I can look up the details of your order?[INST]Certainly, the order ID is f633c884a99eb63866600655356d973e.[/INST]  Hello! I\'m happy to help you with your order information. Your order ID is f633c884a99eb63866600655356d973e. Could you please provide me with more details regarding your inquiry?[INST]I want to know the expected delivery date.[/INST]  Hello! Thank you for reaching out to us regarding your order information. Based on the details provided in your message, here is the information we have on your recent order:\nOrder ID: f63

In [ ]:
getResult("3a874b4d4c4b6543206ff5d89287f0c3","I would like to request a refund, please.", False)

In [ ]:
getResult("3a874b4d4c4b6543206ff5d89287f0c3","Besides the chat, what other ways of contacting the company are available? I would like to know how I can reach out by phone or email if needed.", False)

In [ ]:
getResult("3a874b4d4c4b6543206ff5d89287f0c3","I appreciate your assistance. Have a great day! Goodbye!", False)